# Подключаем библиотеки и загружаем данные

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('CICIDS2017.csv')


In [ ]:
# Если в наборе данных несколько типов аномалий, столбец с метками может содержать строки (например, 'BENIGN', 'DoS', 'DDoS', и т.п.)
# Преобразуем метки в числовой формат
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Label'])

# Отделяем признаки от целевой переменной
X = data.drop(columns=['Label'])
y = data['Label']

# Разбиваем данные на обучающую и тестовую выборки (20% на тест)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Подбираем параметры для классификатора

In [ ]:
def objective(trial):
    # Определяем пространство гиперпараметров
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_uniform('random_strength', 1e-9, 10.0),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0),
        'verbose': 0,  # отключаем вывод обучения
        # Если классификация мультиклассовая, выбираем соответствующую функцию потерь
        'loss_function': 'MultiClass' if len(np.unique(y)) > 2 else 'Logloss'
    }
    
    # Инициализируем модель с подобранными гиперпараметрами
    model = CatBoostClassifier(**params, random_state=42)
    
    # Используем StratifiedKFold для кросс-валидации (3 фолда)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, valid_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                  early_stopping_rounds=50, verbose=False)
        preds = model.predict(X_val)
        acc = accuracy_score(y_val, preds)
        scores.append(acc)
    return np.mean(scores)

# Обучаем CatBoost используя Optuna 

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # число итераций можно увеличить для лучшего поиска

print("Лучшая оптимизация:")
best_trial = study.best_trial
print("  Лучший результат (accuracy):", best_trial.value)
print("  Лучшие гиперпараметры:")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
best_params = best_trial.params.copy()
best_params.update({
    'verbose': 0,
    'loss_function': 'MultiClass' if len(np.unique(y)) > 2 else 'Logloss',
    'random_state': 42
})

final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train, y_train)

# Оценка модели на тестовой выборке

In [ ]:
y_pred = final_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("\nТочность на тестовой выборке: {:.4f}".format(accuracy))

print("\nОтчёт по классификации:")
print(classification_report(y_test, y_pred, target_names=le.classes_.astype(str)))

print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))